<a href="https://colab.research.google.com/github/yeonok93/DL_NLP_news_by_KoBERT/blob/main/1_%EB%89%B4%EC%8A%A4%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 모듈 import, 환경 설정

In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install kobert_transformers

     |████████████████████████████████| 47.3 MB 1.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 14.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595739 sha256=45de796848291d06187e4c40b89083991f0bfb689004fe986bb0a8ce88952476
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 13.8 MB/s 
     |████████████████████████████████| 3.8 MB 12.9 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 895 kB 70.9 MB/s 
     |████████████████████████████████| 6.5 MB 70.5 MB/s 
     |████████████████████████████████| 596 kB 63.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from tqdm import tqdm, tqdm_notebook
import json
from sklearn.model_selection import train_test_split
import os

from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

from transformers import AdamW    #모델의 초기값 함수를 아담으로 지정
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
device = torch.device("cuda:0")   #gpu 사용
# device = torch.device("cpu")
if torch.cuda.is_available():
    print('GPU running')
else:
    print('GPU not running')

GPU running


## 데이터 준비

In [60]:
# text의 길이가 80이상 256이하의 문장 만들기
def text_slicing(text, max_seq_len) :
  text = text.split('\n')
  saved_text, s, i, data = '', 0, 0, []
  for t in text :
    if len(t) < 8 or t[0] in ['o','『', '(', '┌','│', '└', 'ㄴ', '┌','├','◎', '[', '■', 'ㄱ', '-', '.', '<'] : 
      continue
    if t[-1] in ['쪽', '-', '>', ']'] or t[1] in [')'] or t[0:2] in ['vs', '만,', '해!'] :
      continue
    if len(t) > max_seq_len :
        data.append(saved_text)
        data.append(t)
        saved_text, s = '', 0
        i += 1
    elif s + len(t) > max_seq_len :
        data.append(saved_text)
        saved_text, s = t, len(t)
        i += 1
    else :
        saved_text += t
        s += len(t)
  data.append(saved_text)
  
  ret_data = []
  for d in data :
    if len(d) >= 80 :
      ret_data.append(d)
  return ret_data

In [61]:
from google.colab import drive

drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/news_class9x1400'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
max_seq_len = 256
train_dataset = []
train_labels = []

for (path, dir, files) in os.walk(data_path):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.txt':
            with open("%s/%s" % (path, filename), encoding="utf-8") as f:
                label = path[path.rindex('/')+1:]
                text = f.read()
                data = text_slicing(text, max_seq_len)
                train_dataset.extend(data)
                train_labels.extend([label]*len(data))

print('뉴스 카테고리 :', set(train_labels))
len(train_dataset), len(train_labels)

뉴스 카테고리 : {'politic', 'economy', 'sport', 'ITscience', 'health', 'social', 'entertainment', 'culture', 'life'}


(137023, 137023)

In [63]:
train_dataset[0]

'자꾸 읽다가 정들어버린 〈탈북 여대생〉책을 읽는 일, 책을 만드는 일. 오랫동안 꿈꿔왔던 일을 하기 위해 출판사에 발을 내딛었습니다.그리고 첫 출근날, 처음으로 맞이했던 원고는 교정이 거의 완성된 원고였기에 심하게(?) 교정교열을 볼 필요는 없었지만 사소한 맞춤법, 띄어쓰기 등을 교정하였지요.의문나는 점은 저자와 상의하면서 원고를 다듬어 나갔고, 원고를 다 살펴보고 나서는 예쁘게 포장해 줄 카피를 연구하고 보도 자료를 작성했습니다.'

## 전처리

In [64]:
#bert 모델, vocab 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [65]:
df = pd.DataFrame({'content':train_dataset,
                   'label':train_labels})

# dataset의 balance 여부 확인
df.groupby(by=['label']).count()

,content
label,
ITscience,16362
culture,15139
economy,16302
entertainment,15334
health,16004
life,14764
politic,13766
social,14396
sport,14956


In [66]:
label_encoder = LabelEncoder()
label_encoder.fit(df['label'])
num_labels = len(label_encoder.classes_)

df['encoded_label'] = np.asarray(label_encoder.transform(df['label']), dtype=np.int32)
df.tail(3)

,content,label,encoded_label
137020,"궁금해 물으니 꼼수라도 부리는 양 우물쭈물하지 않고 찝찝함 없이 대답하고, 무례하지...",economy,2
137021,공사중인 현장이기에 제한된 공간에 머물 수밖에 없었지만 가족의 보금자리가 시공되는 ...,economy,2
137022,요즘 같은 불경기에 이런저런 정책이나 금융지원도 도움이 되겠지만 건설사들이 자구책으...,economy,2


In [67]:
# Split Train and Validation data
train_texts, test_texts, train_labels, test_labels = train_test_split(df.content, df.encoded_label, test_size=0.2, random_state=123, shuffle=True, stratify=df.encoded_label)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.2, random_state=123, shuffle=True, stratify=train_labels)

print(len(train_texts), len(train_labels))
print(len(val_texts), len(val_labels))
print(len(test_texts), len(test_labels))

87694 87694
21924 21924
27405 27405


In [68]:
train_dataset, val_dataset, test_dataset = [], [], []

for text, label in zip(train_texts, train_labels) :
  train_dataset.append([text, label])

for text, label in zip(val_texts, val_labels) :
  val_dataset.append([text, label])

for text, label in zip(test_texts, test_labels) :
  test_dataset.append([text, label])

len(train_dataset), len(val_dataset), len(test_dataset)

(87694, 21924, 27405)

In [69]:
train_dataset[0]

['납득할 수 없는 이유로 작별을 고하게 됩니다라며 아쉬움을 전했다.마지막으로 허 아나운서는 5천만 소비자의 불만이 사라지는 그 날까지 함께하자던 약속. 그 약속을 결정권자는 기억하지 못하시나봅니다.잊지 말아주세요.수없이 많아진 소비자 고발 프로그램 가운데 그 시작은 MBC가 만들어 낸 불만제로라는 작품이었음을이라며 장문의 글을 끝맺었다.MBC 노동조합 부당하게 폐지당한 <불만제로> 후속 <블랙박스>는 졸속프로그램',
 3]

In [70]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [71]:
# Setting parameters
max_len = 256
batch_size = 16
learning_rate =  5e-5
num_epochs = 10

warmup_ratio = 0.1
max_grad_norm = 1
log_interval = 200

In [86]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [73]:
#BERTDataset 클래스 이용, TensorDataset으로 만들어주기
train_data = BERTDataset(train_dataset, 0, 1, tok, max_len, True, False)
val_data = BERTDataset(val_dataset, 0, 1, tok, max_len, True, False)
test_data = BERTDataset(test_dataset, 0, 1, tok, max_len, True, False)

In [74]:
#배치 및 데이터로더 설정
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=1)

In [82]:
del df
del train_texts
del train_labels
del val_texts
del val_labels
del test_texts
del test_labels

del train_data
del val_data
del test_data
del tok
del tokenizer
del train_dataset
del val_dataset
del test_dataset

## 모델 생성, 학습

In [9]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=9,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        #valid_length 까지만 1, 나머지는 0으로 mask를 생성
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [10]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [75]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [23]:
def train_for_epochs(num_epochs, train_dataloader, test_dataloader) :
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]

    optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5) 
    loss_fn = nn.CrossEntropyLoss()

    t_total = len(train_dataloader) * num_epochs
    warmup_step = int(t_total * warmup_ratio)

    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
  
    # train
    for e in range(num_epochs):
        train_acc = 0.0
        test_acc = 0.0
        # training
        model.train()

        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
            optimizer.zero_grad()
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)
            loss = loss_fn(out, label)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            scheduler.step()  # Update learning rate schedule
            train_acc += calc_accuracy(out, label)
          
            if batch_id % log_interval == 0:
                print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
        print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
        model.eval()
        
        # validation
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
            token_ids = token_ids.long().to(device)
            segment_ids = segment_ids.long().to(device)
            valid_length= valid_length
            label = label.long().to(device)
            out = model(token_ids, valid_length, segment_ids)
            test_acc += calc_accuracy(out, label)
        print("epoch {} validation acc {}".format(e+1, test_acc / (batch_id+1)))

In [11]:
def test(test_dataloader) :
  out_li = []
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
      {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
      {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
  ]

  optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5)  # lr = learning rate
  loss_fn = nn.CrossEntropyLoss()

  t_total = len(train_dataloader) * num_epochs
  warmup_step = int(t_total * warmup_ratio)
  test_acc, test_loss = 0.0, 0.0
  model.eval()

  scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)
    valid_length= valid_length
    label = label.long().to(device)
    out = model(token_ids, valid_length, segment_ids)
    out_li.append(out)
    test_acc += calc_accuracy(out, label)
    test_loss += loss_fn(out, label)
  print("Test acc {}, loss {}".format(test_acc / (batch_id+1), test_loss))
  return out_li

In [78]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
train_for_epochs(num_epochs, train_dataloader, val_dataloader)

In [ ]:
torch.save(model, '/content/drive/MyDrive/models/news_category_256_16_5e-5_10.pt')

In [88]:
gc.collect()
torch.cuda.empty_cache()

In [ ]:
test(test_dataloader)

## 모델 평가
  -네이버 기사에서 카테고리별로 10개씩 가져와 테스트

In [45]:
def softmax(vals, idx):
    valscpu = vals.cpu().detach().squeeze(0)
    a = 0
    for i in valscpu:
        a += np.exp(i)
    return ((np.exp(valscpu[idx]))/a).item() * 100

In [46]:
loaded_model = torch.load('/content/drive/MyDrive/models/news_category_256_16_5e-5_10.pt')
loaded_model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [47]:
def testModel(model, seq):
    category = ["IT/과학","문화","경제", "연예","건강","생활", "정치", "사회", "스포츠"]
    tmp = [seq]
    transform = nlp.data.BERTSentenceTransform(tok, max_len, pad=True, pair=False)
    tokenized = transform(tmp)

    loaded_model.eval()

    result = model(torch.tensor([tokenized[0]]).to(device), [tokenized[1]], torch.tensor(tokenized[2]).to(device))
    idx = result.argmax().cpu().item()
    print(f'뉴스의 카테고리 : {category[idx]} (신뢰도 {round(softmax(result,idx), 2)}%)')
    return category[idx]

In [48]:
#정치
politic = ['6·1 지방선거 공천을 두고 고심 중인 더불어민주당에서 29일 ‘이재명 상임고문 역할론’이 제기됐다. 대선에서 분패한 이 고문이 서울시장이나 경기지사에 나서야한다는 여론이 당 일각에서 나온 것이다.',
        '한편, 김 대표는 이날 국회에서 “민주당이 제안한 ‘정치교체를 위한 정치개혁 추진기구’ 구성과 합당 제안을 수용한다”고 밝혔다. 경기지사 출마 가능성에 대해선 “가장 빠른 시간 내에 의견을 듣고 결정하겠다”고 했다.',
        '김윤덕 의원은 공약과 관련해 200만 도민, 3% 연평균 경제성장률, 4만불 1인당 지역총소득(GRDP) 등 \'2·3·4비전\'을 제시했다.',
        '김경수 전 지사의 핵심 프로젝트인 \'부울경 메가시티\'와 관련해 "부울경이 한 몸이 돼 수도권에 대항하는 취지에는 동의하지만, 여기에 대도시만 포함되면 소멸 위기에 처한 지역은 소외 가능성이 있다"며 "도민 여론을 수렴해 신중히 접근해야 한다"는 반응을 보였다.',
        '대구시장에 출마하는 홍준표 국민의힘 의원은 29일 "100m달리기를 하는데 (상대방은) 10m를 뛰어주고 하는 경기가 공정과 상식에 부합하느냐"고 비판했다.',
        '한편 국민의힘 6·1지방선거 공천관리위원회는 29일 논란이 일었던 중복 페널티 25%를 1인당 최대 10%로 변경하기로 했다. 이에 따라 대구시장에 출마하는 홍준표 국민의힘 의원은 페널티 최대 10%만 받을 것으로 보인다.',
        '청와대가 문재인 대통령 부인 김정숙 여사의 ‘옷값 논란’에 대해 입장을 밝힌 가운데, 박근혜 전 대통령의 최측근인 유영하 변호사는 박 전 대통령의 의상비 논란 관련 “역사가 밝혀줄 것”이라고 말했다.',
        '박 전 대통령 의상비 관련 정보공개 소송 당시 청와대는 ‘개인이 부담했다’고 밝혔지만, 관련 검찰 조사에서 박 전 대통령이 전직 국정원장들로부터 33억 원을 받은 것으로 드러났고 법원은 그 일부가 의상실 유지비 등으로 사용됐다고 판단했다.',
        '해당 문건에서는 \'출근길 시위 잠시 휴전 상태지만 디테일한 약점은 계속 찾아야 한다\'며 고의적인 열차 운행 방해를 증빙하라는 등 대응 방법을 언급하고 있다.',
        '대통령직인수위원회가 추진 중인 \'임대차3법(계약갱신청구권·전월세 상한제·전월세 신고제) 폐지·축소\'에 대해 거대 야당인 더불어민주당이 반대 입장을 내놓으면서 국회 논의 과정에서 가시밭길이 예상된다. 여소야대 상황을 차치하더라도 이미 시행된 임대차3법의 전면 개편을 두고 시장 충격이 적지 않을 것이란 우려도 제기되는 상황이다.']
acc = 0
for sentence in politic:
    if testModel(loaded_model, sentence) == '정치':
        acc += 1
print(f'\n정확도 : {acc*100 // len(politic)}%')

뉴스의 카테고리 : 정치 (신뢰도 99.96%)
뉴스의 카테고리 : 정치 (신뢰도 99.97%)
뉴스의 카테고리 : 정치 (신뢰도 99.96%)
뉴스의 카테고리 : 정치 (신뢰도 99.96%)
뉴스의 카테고리 : 정치 (신뢰도 99.7%)
뉴스의 카테고리 : 정치 (신뢰도 99.95%)
뉴스의 카테고리 : 연예 (신뢰도 67.27%)
뉴스의 카테고리 : 정치 (신뢰도 76.07%)
뉴스의 카테고리 : 사회 (신뢰도 99.07%)
뉴스의 카테고리 : 경제 (신뢰도 99.98%)

정확도 : 70%


In [49]:
#경제
economy = ['김부겸 국무총리는 30일 "서대구역이 대구·경북의 발전은 물론이고 국가균형발전과 동서 화합을 이어가는 \'희망 이음\'의 중요한 고리가 되기를 진심으로 바란다"고 말했다. 김 총리는 이날 서대구역 개통식에 참석, "서대구역은 대구와 광주를 연결하는 \'달빛내륙철도\', \'광주대구선\'의 출발역"이라며 이같이 말했다.',
           '전국경제인연합회(전경련) 부설 한국경제연구원(한경연)이 대통령 집무실을 현 청와대에서 용산 국방부 청사로 옮길 경우 경제적 효과가 최대 5조1천억원에 달할 것이라는 보고서를 내놨다. 일각에서는 문재인 정부에서 철저하게 \'패싱\'당했던 전경련이 윤석열 당선인 공약에 적극 힘을 실어주는 방식으로 위상 되찾기에 나섰다는 해석이 나온다.',
           '금융 당국이 코스닥 상장사인 에디슨EV(136510)의 ‘주가조작’ 혐의 조사에 착수했다. 에디슨EV는 쌍용차(003620) 인수를 추진했던 에디슨모터스의 자회사로 지난해 에디슨모터스가 쌍용차 인수에 뛰어들면서 에디슨EV 주가는 급등했다. 이 과정에서 대주주 투자조합이 주식 대부분을 처분하고 차익 시현에 나서며 ‘먹튀’ 논란이 빚어졌다. 금융 당국은 에디슨모터스의 쌍용차 인수가 무산으로 결론 나면서 ‘주가조작’ 논란이 의혹 수준을 넘어섰다고 보고 있다. 에디슨모터스가 인수 능력이 없는데도 주가 시세차익을 노리고 인수전에 뛰어든 것은 아닌지 집중적으로 살필 예정이다.',
           '박정호 SK하이닉스 부회장은 30일 영국 반도체 설계 전문기업 \'ARM\'을 공동으로 인수하는 방안을 검토 중이라고 밝혔다. 박 부회장은 이날 경기도 이천 본사에서 열린 정기 주주총회 직후 \'ARM 인수를 검토하냐\'는 기자들의 질문에 "ARM은 한 회사가 인수할 수 있는 기업은 아니라고 생각한다"며 "전략적 투자자들과 함께 컨소시엄으로 인수하는 방안을 검토 중"이라고 답했다.',
           '서울시가 예상보다 빠르게 광주 학동 철거건물 붕괴사고와 관련해 행정처분을 내리면서 화정아이파크 외벽 붕괴사고에 대한 처분 시기에도 관심이 모아지고 있다. 서울시는 30일 광주 학동참사와 관련해 HDC현대산업개발에 8개월의 영업정치 처분을 내렸다. HDC현산 관계자는 이와 관련 "광주 사고 수습을 위해 최선을 다하겠다는 입장에는 변함이 없다"며 "향후에도 직원과 협력사, 주주, 고객 등 이해관계자들을 위해 신중하게 고민하고, 수습에 만전을 기하겠다"고 밝혔다.',
           '최근 일본 증시가 강하게 반등하고 있다. 니케이225지수는 지난 9일 이후 14% 넘게 상승하며 단숨에 2만4000대에서 2만8000선 위로 뛰어올랐다. 2700선을 버겁게 사수하며 부진한 흐름을 보이고 있는 코스피지수와 비교되는 모습이다.',
           '사업보고서 제출 기한이 하루 앞으로 다가온 가운데 30곳이 넘는 상장사들이 아직까지 감사보고서를 미제출한 것을 나타났다. 사업보고서 제출 지연은 상장폐지 사유가 되는 만큼 투자자들의 각별한 유의가 필요하다는 지적이다.',
           '카카오뱅크는 지난달 22일 출시한 비대면 주택담보대출 잔액이 1000억원을 넘어섰다고 30일 밝혔다. 신규 주택구입자금과 기존 주담대 대환, 생활안정자금, 전월세보증금 반환 등 모든 주담대를 포함한 금액이다. 이 중 신규 주택구입 목적의 주담대 약정 비율은 35%이며, 평균 금리는 연 3.69%로 나타났다. 지난 1월 기준 시중은행의 주택담보대출 가중평균금리(연 3.85%)보다 소폭 낮은 수준이다.',
           '쌍용차(003620) 매각이 사실상 무산되면서 문재인 정부와 KDB산업은행이 추진했던 기업 구조조정 작업이 도마 위에 올랐다. 문재인 정부가 매각을 통해 민영화 혹은 정상화를 추진했던 기업 중 성공한 사례는 대우건설(047040)이 유일하다. 대우건설 역시 투입된 공적자금 중 1조원가량을 회수하지 못하면서 ‘저가 매각’이라는 지적이 나왔다.',
           '오는 4월 1일부터 전국 카페·음식점 안에서는 일회용품 사용이 금지된다. 일회용품을 사용하다가 적발될 경우 50만~200만원 과태료가 부과된다. 단 일회용품 사용 금지 규제는 매장에서만 적용하며, 배달이나 테이크아웃의 경우엔 사용을 허용한다.'
]

acc = 0
for sentence in economy:
    if testModel(loaded_model, sentence) == '경제':
        acc += 1
print(f'\n정확도 : {acc*100 // len(economy)}%')

뉴스의 카테고리 : 정치 (신뢰도 99.93%)
뉴스의 카테고리 : 정치 (신뢰도 96.59%)
뉴스의 카테고리 : 경제 (신뢰도 99.63%)
뉴스의 카테고리 : IT/과학 (신뢰도 99.99%)
뉴스의 카테고리 : 경제 (신뢰도 95.87%)
뉴스의 카테고리 : 경제 (신뢰도 99.99%)
뉴스의 카테고리 : 경제 (신뢰도 99.99%)
뉴스의 카테고리 : 경제 (신뢰도 99.99%)
뉴스의 카테고리 : 경제 (신뢰도 99.99%)
뉴스의 카테고리 : 건강 (신뢰도 99.96%)

정확도 : 60%


In [50]:
#사회
society = ['오는 4월 1일부터 전국 카페·음식점 안에서는 일회용품 사용이 금지된다.',
           '질병관리청 중앙방역대책본부(방대본)는 2일 0시 기준 신종 코로나바이러스 감염증(코로나19) 신규 확진자가 26만4171명 발생했다고 밝혔다. 이 중 국내 지역발생은 26만4103명, 해외유입은 68명을 기록했다.',
           '지난 겨울부터 전국장애인차별철폐연대(이하 전장연)의 출근길 지하철 타기 행동이 시작된 이후, 장애인 이동권 이슈가 그 어느 때보다 뜨겁다. 3월 17일 YTN 보도로, 서울교통공사가 “사회적 약자와의 여론전 맞서기-전국장애인차별철폐연대 지하철 시위를 사례로”라는 문건을 작성해 장애인 지하철 시위에 대한 부정적 여론을 조성하려 한 정황이 드러나 많은 이들의 공분을 샀다.',
           '서울시 시각장애인 콜택시 차량이 3년간 한대도 늘지 않은 것으로 나타났다. 시각장애인이 콜택시를 이용하려면 40분 이상이 소요된다는 분석이다. 1일 국민의힘 김예지(사진) 의원실에 따르면 서울시 시각장애인 복지콜 서비스 운행 차량은 최근 3년 동안 증차 없이 158대를 유지하고 있다.',
           '질병관리청 중앙방역대책본부(방대본)는 2일 0시 기준 신규 사망자 339명 중 10세 미만이 2명이라고 밝혔다. 누적 사망자는 13명으로, 전연령 누적 사망 1만6929명 대비 0.1% 수준이다.',
           '정부는 2주간 코로나19 유행이 확연히 감소세로 전환하고, 위중증 환자와 의료체계가 안정적인 수준을 보인다면, \'실내 마스크 착용\' 등 핵심수칙을 제외한 영업시간, 사적모임, 대규모 행사 등 모든 조치 해제를 검토하겠다고 밝혔다.',
           '정부가 이달 중 코로나19 사망자의 장례 관련 고시를 폐지하겠다고 밝혔지만 시신을 통한 감염 전파를 우려하는 현장의 분위기가 바뀌지 않으면 유족들의 장례 등이 원활히 이뤄지는 데까지는 제한적일 수 있다는 지적이 나온다.',
           '코로나19 유행 대책을 논의해 온 중앙재난안전대책본부 회의도 주 2회로 축소 개최하기로 하는 등 코로나19의 계절독감화를 위한 작업에 시동을 걸었다. 다만 의료계에서는 위중증 환자·사망자 증가 본격화에 대응할 수 있는 정교한 의료대응을 요구하는 목소리도 커지고 있다.',
           '전장연 측은 이날 면담에서 인권위의 적극적인 역할을 촉구했다고 한다. 박경석 전장연 대표는 “이번 시위에 대해 국민의힘 이준석 대표가 발언한 내용과 표현 방식은 명확한 인권침해”라며 “인권위에 이 문제를 지적하는 공식 입장을 내달라고 요청했다”고 했다. 앞서 이준석 대표는 장애인들의 이번 시위를 두고 “시민을 볼모로 하는 불법 투쟁”, “순환선 2호선은 후폭풍이 두려워서 못 건드린다” 등의 발언을 한 바 있다.',
           'A씨는 경찰의 황당한 단속에 억울함을 참지 못해 잠도 못 자고 다음날 오전 블랙박스 영상을 확보해 해당 지역인 계양경찰서 민원실을 방문했다. A씨는 "그곳에서 무고함을 확인했고 벌금과 벌점을 취소 받았다"면서 "그날 해당 경찰에게 \'미안하다\'는 전화가 왔고, \'커피라도 한잔 사드리겠다\'고 하더라"고 했다.']

acc = 0
for sentence in society:
    if testModel(loaded_model, sentence) == '사회':
        acc += 1
print(f'\n정확도 : {acc*100 // len(society)}%')

뉴스의 카테고리 : 건강 (신뢰도 99.97%)
뉴스의 카테고리 : 사회 (신뢰도 95.1%)
뉴스의 카테고리 : 사회 (신뢰도 98.89%)
뉴스의 카테고리 : 생활 (신뢰도 97.6%)
뉴스의 카테고리 : 사회 (신뢰도 99.96%)
뉴스의 카테고리 : 건강 (신뢰도 99.97%)
뉴스의 카테고리 : 건강 (신뢰도 99.89%)
뉴스의 카테고리 : 건강 (신뢰도 99.97%)
뉴스의 카테고리 : 사회 (신뢰도 99.97%)
뉴스의 카테고리 : 사회 (신뢰도 99.96%)

정확도 : 50%


In [51]:
#생활
life = ['경복궁 야간 관람이 봄을 맞아 시작한다. 문화재청 궁능유적본부 경복궁관리소는 \'2022년 상반기 경복궁 야간 관람\'을 1일부터 5월29일의 기간 중 38일간 개최한다고 밝혔다. 경복궁 야간 관람은 매년 예매 시작과 동시에 매진될 만큼 큰 인기를 누리고 있는 대표적인 궁궐 활용 프로그램으로, 특히, 상반기 야간 관람에서는 봄꽃이 만개한 경복궁의 야경을 감상할 수 있다.',
        '가전 구매 혜택은 물론 혼수, 입주를 준비 중인 고객이라면 제휴 브랜드 혜택과 최대 50만 포인트 혜택도 받아볼 수 있다. 추가적으로 행사 기간 동안 가전을 구매한 고객 중 1,000명을 추첨하여 3,000만원 상당의 여행 경품을 제공한다.',
        '긴 겨울이 지나고 봄이 오면 춘곤증이 온다. 몸이 나른하고 피곤해진다. 이럴 때는 음식으로 우리 몸을 돌보아 주어야 한다. 특히 요즈음 오미크론이라는 전염병 확진자가 주변에 너무 많이 나오고 있다. 먹는 음식을 잘 섭취해 몸의 면역력을 갖는 일이 무엇보다 중요해졌다. ',
        '학교 정문 입구에 아이들이 모여 있었다. 무슨 일인가 싶어 가까이 가 보니 나무에 트리 장식을 하고 있는 것이 아닌가. 뭐하는 것이냐고 물으니 아이들은 "그냥이요"라고 답하며 웃기만 했다. 그때까지도 몰랐다. 먼저 나온 선생님도 같은 모습을 본 것 같았다. 만우절이라고 했다. 아이들이 뭔가는 하는 모양이라며. 아하! 미소가 지어졌다. 만우절이라고 철을 건너뛴 장난을 하다니. 그게 하필 4월의 크리스마스라니. ',
        '올해 서울의 주요 벚꽃길이 3년 만에 전면 개방된다. 여의도 윤중로와 잠실 석촌호수, 양재천 등 벚꽃 명소를 찾으려는 상춘객들의 기대감은 커지고 있다. 서울 벚꽃 개화시기가 당초보다 일주일 가량 늦어졌지만, 오랜만에 벚꽃길을 거닐면서 마음껏 풍경을 감상할 수 있게 됐다.',
        '4월의 첫 주말인 2일 대구와 경북지역은 대체로 맑고 따뜻하겠습니다. 낮 최고기온은 14~18도 분포로 어제보다 3~4도 가량 오르겠습니다. 3일도 대체로 맑은 가운데 아침 기온은 영하 3도에서 영상 6도, 낮 기온은 14~19도 분포로 오늘과 비슷하겠습니다. ',
        '전남 구례군 산동면은 ‘산수유 마을’로 불린다. 이른 봄이면 49개 마을 전체가 노랗게 물드는 전국 최대 산수유 군락지다. 코로나 탓에 2020년부터 3년 연속 산수유 봄 축제가 취소됐지만 요즘 봄의 전령사 산수유를 만끽하려는 상춘객의 발걸음이 끊이질 않는다. 개나리, 진달래, 벚나무, 철쭉보다 일찍 꽃을 피우는 산수유는 봄엔 노란 꽃을, 가을엔 빨간 열매를 내놓는다. 조경수로도 좋지만 한약재와 건강식품 재료로 사용하는 열매를 얻으려고도 많이 키운다.',
        '롯데월드가 33년간 테마파크 운영 경험을 살려 부산에서 처음으로 퍼레이드를 시작한다. 2013년 광안리 미월드가 폐업한 지 10년 만에 부산 중심부에 롯데월드 어드벤처 부산이 문을 연다. 교통 혼잡과 소음 등 민원 우려가 적지 않지만 부산 사람들은 봄꽃 지천인 이 계절에 알록달록 의상을 갖춰 입은 공연자들의 퍼레이드를 처음으로 집 가까운 곳에서 즐길 수 있게 됐다.',
        '미국의 유명 신경과학자 조지프 르두(Joseph E. LeDoux)는 EBS의 강연 시리즈 \'위대한 수업 GREAT MINDS\'에서 이렇게 이야기했다. “1949년 이후 모든 세대가 자신들이 가장 불안하다고 여겨왔다.” 공포의 뇌과학으로 유명한 그는 인간이 느끼는 불안이 사실은 매우 당연한 현상이라고 말한 것이다. 세계적인 학자의 주장에 따르자면 한국인들, 특히 청년세대가 겪고 있는 심리적 어려움도 전혀 특별한 것이 아니게 된다.',
        '"이 복잡 미묘한 문제의 한가운데에서 \'육식\'만 단순히 악역을 맡아 집중포화를 맞고 있는 건 아닌가? \'고기\'를 먹지 않는 것이 과연 이 모든 문제의 합당한 해결책인가?" 최근 트렌드를 이해할 때 \'비건\'은 빼놓을 수 없는 키워드다. 엄격한 의미의 비건이 아니더라도, 기후위기와 동물복지 등 윤리적 이슈, 건강과 미용 등 라이프스타일 이슈로 채식은 다양한 관심을 받고 있다.']

acc = 0
for sentence in life:
    if testModel(loaded_model, sentence) == '생활':
        acc += 1
print(f'\n정확도 : {acc*100 // len(life)}%')

뉴스의 카테고리 : 생활 (신뢰도 99.95%)
뉴스의 카테고리 : IT/과학 (신뢰도 99.98%)
뉴스의 카테고리 : 건강 (신뢰도 99.97%)
뉴스의 카테고리 : 생활 (신뢰도 99.86%)
뉴스의 카테고리 : 스포츠 (신뢰도 99.88%)
뉴스의 카테고리 : 생활 (신뢰도 99.86%)
뉴스의 카테고리 : 생활 (신뢰도 99.96%)
뉴스의 카테고리 : 문화 (신뢰도 99.96%)
뉴스의 카테고리 : 건강 (신뢰도 99.8%)
뉴스의 카테고리 : 건강 (신뢰도 99.97%)

정확도 : 40%


In [52]:
#연예
entertain = ['가수 케빈 오(32)가 배우 공효진과 열애를 직접 인정했다. 그는 공효진과 열애가 알려진 1일 팬카페를 통해 열애를 공식화했다. 케빈 오는 공효진과 열애가 알려진 이날 오후 자신의 공식 팬클럽 \'케빈 오의 다락방\'에 "안녕하세요 케빈오입니다"라는 제목으로 장문의 글을 남겼다.',
             '홍상수, 김민희 불륜 커플의 국내 한정 두문 불출 행보가 길어지고 있다. 국제 영화제 다정히 트로피를 거머쥐던 모습을 국내에선 볼 수 없다. 작품을 내놓으면서도 행사에는 일절 참여하지 않고 있다.',
             '소속사 큐브엔터테인먼트는 1일 “홍석은 오는 2022년 5월 3일 훈련소에 입소할 예정”이라고 밝혔다. 큐브 측은 “코로나19 확산 방지를 위해 입대 장소와 시간은 비공개이며 특별한 절차 없이 조용히 입소할 예정”이라며 “국방의 의무를 성실히 수행하고 건강한 모습으로 돌아올 펜타곤 홍석에게 따뜻한 응원 부탁드린다”고 했다.',
             '방송인 정형돈이 아이들을 노출하지 않는 이유에 대해 설명했다. 1일 방송된 채널A ‘오은영의 금쪽상담소’에서는 핫한 \'전 부부\' 유튜버 최고기, 유깻잎이 오은영 박사와 상담을 하며 고민을 이야기하는 시간을 가졌다.',
             '1일 첫 방송된 MBC 금토드라마 ‘내일’에서 최준웅(로운 분)이 위기관리팀 계약직 사원이 되는 모습이 그려졌다. 이날 방송에서 최준웅은 \'탁 비료컴퍼니\'에 최종 면접을 보게 됐고, 이후 탈락 문자를 받게 된다. ',
             '한창 멤버들의 이상형을 이야기하던 현장에 한가인이 출몰, 멤버들은 크게 놀란다.출연 사실을 전혀 몰랐던 연정훈은 "여보 여기 왜 왔어"라며 \'찐\' 리액션으로 당황함을 내비치지만 이내 안으며 반겨준다.한가인은 연정훈을 속이기 위해 촬영 당일 아침까지도 007 작전을 방불케 했던 비하인드스토리를 공개해 연정훈을 또 한 번 놀라게 한다.',
             '배우 현빈, 손예진이 결혼식 현장에서 서로에게 쓴 편지를 낭독했다. 2일 한 사회관계망서비스(SNS) 계정에는 현빈과 손예진이 결혼식 중에 서로에게 쓴 편지를 읽는 모습이 담긴 영상을 올랐다. 공개된 영상에서 마주한 현빈과 손예진은 차례로 서로에게 편지를 읽어주는 모습이다. 손예진은 "자기야 안녕? 오늘은 드디어 우리의 결혼식 날이네, 아직도 믿기지가 않아"라고 말하며 결혼식의 설렘을 드러냈다.',
             '방탄소년단은 오는 8~9일(이하 현지시간)과 15~16일 라스베이거스 얼리전트 스타디움(Allegiant Stadium)에서 \'BTS PERMISSION TO DANCE ON STAGE - LAS VEGAS\'를 개최한다. 지난달 서울 잠실종합운동장 주경기장에서 열린 \'BTS PERMISSION TO DANCE ON STAGE - SEOUL\' 이후 약 한 달 만에 대면 콘서트를 통해 다시 팬들을 만난다.',
             '1일(어제) 방송된 MBC ‘나 혼자 산다’에서 박나래는 대식가와 소식가 손님들을 집에 초대해 ‘나래 미식회’를 열어 안방극장의 오감을 만족시켰다. 박나래는 셰프복까지 차려입고 등장, 손님들이 도착하기 전 분주하게 식재료를 손질하며 설렘을 드러냈다. “상다리가 부러지는 걸 보여주겠다”라고 야심 차게 팔을 걷어붙인 박나래는 전날 국물 요리를 먹어서 손에도 살이 찐 것 같다며 라텍스 장갑을 끼면서 괴로워하는 등 심상치 않은 예능감을 뽐냈다.',
             '이이경은 1일 방송된 E채널 신규 예능 프로그램 \'용감한 형사들\' 1회에서 범죄 사건에 대한 이해도와 집중력을 드높이는 \'사건 스토리텔러\'로 대활약하며 \'범죄 콘텐츠 예능\'에 손쉽게 다가갈 수 있도록 가교역할을 톡톡히 해냈다.']

acc = 0
for sentence in entertain:
    if testModel(loaded_model, sentence) == '연예':
        acc += 1
print(f'\n정확도 : {acc*100 // len(entertain)}%')

뉴스의 카테고리 : 연예 (신뢰도 99.95%)
뉴스의 카테고리 : 연예 (신뢰도 99.93%)
뉴스의 카테고리 : 연예 (신뢰도 99.98%)
뉴스의 카테고리 : 연예 (신뢰도 99.71%)
뉴스의 카테고리 : 연예 (신뢰도 99.98%)
뉴스의 카테고리 : 연예 (신뢰도 99.98%)
뉴스의 카테고리 : 연예 (신뢰도 99.98%)
뉴스의 카테고리 : 문화 (신뢰도 99.9%)
뉴스의 카테고리 : 연예 (신뢰도 99.98%)
뉴스의 카테고리 : 연예 (신뢰도 99.97%)

정확도 : 90%


In [53]:
#스포츠
sports = ['한국이 2022 국제축구연맹(FIFA) 카타르 월드컵에서 죽음의 조를 피했다. 상대적으로 무난한 조다. 반면 일본은 스페인, 독일과 한 조에 속하며 \'죽음의 조\'에 속했다. 4월 2일 오전 1시(한국시간) 카타르 도하에 위치한 도하 국립컨센션센터(DECC)에서 2022 카타르 월드컵 본선 조별리그 조추첨이 개최됐다. 조추첨을 통해 8개 그룹에 4팀씩 편성되고, 12일간 조별리그를 펼친 뒤 각 조 상위 2팀이 16강 토너먼트로 진출한다.',
          '올해 프로야구는 그 어느 때보다 대형신인들이 풍성한 한 해가 될 것 같다. KBO는 지난 1일 ‘2022 신한은행 SOL KBO리그’ 정규시즌 개막전을 앞두고 개막전 엔트리를 발표했다. 올해는 11명의 신인선수가 개막전 엔트리에 이름을 올렸다.',
          '에릭 텐 하흐 감독과 아약스의 이별이 가까워지고 있다. 영국 매체 \'스카이스포츠\'는 2일(한국시간) "텐 하흐 감독이 본인의 이적 가능성을 직접 시사했다. 그는 개인 커리어 다음 단계를 위해, 아약스가 이해해주기를 바란다고 말했다"라고 밝혔다.',
          '다소 충격적인 고백이다. 바르셀로나 수비수 헤라르드 피케가 과거 맨체스터 유나이티드에서 뛰던 시절 음주운전 단속을 피했던 일을 회상했다. 피케는 바르셀로나 유스 시스템이 낳은 스타다. 파트너 카를레스 푸욜과 \'통곡의 벽\'을 구축하며 팀의 수비를 책임졌다. 기량을 만개한 이후에는 매 시즌 스페인 프리메라리가 30경기 이상을 소화하며 팀의 핵심 수비수로 활약했고, 피케의 탄탄한 수비 덕에 바르셀로나는 리그 우승 8회, 코파 델 레이 우승 7회, 챔피언스리그 우승 3회 등의 금자탑을 쌓았다.',
          '42살 베테랑 미들블로커의 마지막 불꽃이 한국도로공사의 추운 겨울을 따듯하게 만들었다. 녹슬지 않은 실력으로 여전히 코트 위에서 본인의 존재감을 드러내며 최고의 시즌을 보냈다. 팀은 역대 최다 12연승을 세웠을 뿐만 아니라 오랜만에 상위권에 이름을 올렸다. 자신의 배구 인생에 5세트 14대 14를 맞이한 정대영. 경기를 마무리 짓는 연속 득점을 얻기 위해 코트 위에서 마지막 투혼을 불태운다.',
          '흥국생명 핑크스파이더스 구단은 1일 보도자료를 통해 박미희 감독의 뒤를 이어 팀을 이끌 새 사령탑으로 전 KB손해보험 스타즈 감독이었던 권순찬 감독을 선임했다고 발표했다. 권순찬 감독은 "선수들과 힘을 합쳐 4회 통합우승에 빛나는 흥국생명의 전통을 되살려 보겠다"며 "배구도 사람이 하는 일이라 감독인 저부터 앞장서 선수와 코칭스태프가 모두 한마음 한 뜻이 될 것"이라고 각오를 밝혔다.',
          '젠지 e스포츠가 LCK 첫 우승에 도전한다. 전신이라고 할 수 있는 삼성 갤럭시까지 올라간다면 2014년 리그 오브 레전드 챔피언스(LoL 챔피언스) 스프링 이후 15시즌 동안 한 번도 우승하지 못했다. 젠지는 2일 오후 5시 경기도 고양시 일산서구에 위치한 KINTEX(킨텍스) 제1전시장 5홀에서 벌어질 예정인 2022 LCK 스프링 결승전서 T1을 상대로 우승에 도전한다.',
          'T1은 LCK 역사상 가장 많은 스플릿 우승을 달성한 팀이다. 2013년 LCK 서머 결승전에서 KT 롤스터 불리츠를 상대로 첫 우승을 달성한 이래, 2015년 스프링부터 2016년 스프링까지, 2019년 스프링부터 2020년 스프링까지 두 번이나 3연속 우승을 차지하는 등 무려 9번이나 정상에 올랐다.',
          '올 시즌 프로야구 선수들은 경미한 부상에도 더욱 각별해야 한다. 한국도핑방지위원회(KADA)가 2022시즌부터 도핑방지규정을 강화해 지난해까지 허용됐던 스테로이드 주사 치료를 금지시켰기 때문이다. 가벼운 허리 통증 부상을 당한 LG 홍창기가 바뀐 규정으로 인해 주사 치료를 받지 못해 개막전 출장이 무산됐다.',
          '출범 40주년을 맞은 2022시즌 프로야구가 2일 오후 2시 잠실 등 전국 5개 구장에서 열리는 개막전을 시작으로 6개월간의 정규리그 레이스에 들어간다. 팀당 144경기, 총 720경기를 치르는 대장정이다. 신종 코로나바이러스 감염증 팬데믹 여파로 지난 두 시즌 동안 관중 입장 인원이 수용 규모의 10∼50%로 제한됐었지만 올 시즌 개막전엔 100% 입장할 수 있다. 육성 응원은 계속 금지되지만 치킨과 맥주 등을 먹고 마시며 관전할 수 있다.']

acc = 0
for sentence in sports:
    if testModel(loaded_model, sentence) == '스포츠':
        acc += 1
print(f'\n정확도 : {acc*100 // len(sports)}%')

뉴스의 카테고리 : 스포츠 (신뢰도 99.99%)
뉴스의 카테고리 : 스포츠 (신뢰도 99.99%)
뉴스의 카테고리 : 스포츠 (신뢰도 99.99%)
뉴스의 카테고리 : 스포츠 (신뢰도 99.98%)
뉴스의 카테고리 : 스포츠 (신뢰도 99.99%)
뉴스의 카테고리 : 스포츠 (신뢰도 98.96%)
뉴스의 카테고리 : IT/과학 (신뢰도 93.95%)
뉴스의 카테고리 : 스포츠 (신뢰도 99.99%)
뉴스의 카테고리 : 스포츠 (신뢰도 99.99%)
뉴스의 카테고리 : 스포츠 (신뢰도 99.99%)

정확도 : 90%


In [54]:
#IT/과학
it_sci = ['디스플레이 내장형 지문인식 스캐너가 달린 새로운 ‘아이폰’ 시리즈가 향후 2년간 출시되지 못할 것이란 전망이 나왔다. 1일 해외 IT매체 GSM아레나에 따르면 애플 분석가인 밍치궈 대만 TF인터내셔널 애널리스트는 최근 자신의 트위터를 통해 “언더디스플레이 터치ID를 탑재한 아이폰 시리즈는 오는 2025년까지 출시되지 않을 것”이라고 밝혔다.',
          '삼성전자가 자사 폴더블(접이식) 스마트폰 \'갤럭시 Z 폴드3\'와 \'갤럭시 Z 플립3\' 제품명에서 러시아군의 상징으로 여겨지는 \'Z\' 지우기 움직임을 넓혀가고 있다. 1일 삼성전자 우크라이나 온라인 판매 페이지를 보면 폴더블폰 제품군인 \'갤럭시Z\' 시리즈 제품명에서 Z가 모두 삭제됐다.',
          '3일이면 한국이 세계 최초로 5G(5세대) 이동통신을 상용화한 지 만 3년이 된다. 5G는 코로나19 팬데믹과 비대면 시대 핵심 인프라로 자리잡았다. 그럼에도 논란은 \'현재진행형\'이다. 당초 기대에 못 미친 속도·품질·커버리지는 소비자 불신으로 이어졌다. 5G 현주소와 개선과제를 짚어본다.',
          '넥슨의 PC·콘솔용 대전 격투 게임 ‘던파 듀얼(DNF Duel)’이 2차 오픈 베타 테스트에 나선다. 넥슨은 2일부터 4일까지 플레이스테이션(PS)4·5 콘솔 플랫폼에서 한국, 일본, 북미, 유럽 등 글로벌 이용자를 대상으로 던파 듀얼의 2차 오픈 베타 테스트를 진행한다고 2일 밝혔다.',
          '구글은 이날부터 자사의 애플리케이션(앱) 마켓 \'구글플레이\'에 새로운 결제정책을 적용해 앱 개발사들에게 구글플레이 인앱결제 또는 인앱결제 내 제3자결제 방식만 허용한다고 밝혔다. 아웃링크 방식의 외부결제 방법은 금지된다. 인앱결제 방식은 최대 30%, 제3자결제 방식은 최대 26%의 수수료가 부과된다.',
          '붉은색 몸통에 반점이 촘촘히 박혀 마치 딸기처럼 보이는 희귀 오징어가 카메라에 포착됐다. 최근 미국 몬터레이만 아쿠아리룸 연구소(MBARI)는 캘리포니아 해안에서 가장 깊은 해저 협곡 중 하나인 몬터레이 협곡에서 희귀 오징어를 발견했다며 해당 영상을 공개했다.',
          '옛것을 되살리는 고고학과 미래를 바라보는 첨단 기술이 만났다. 로봇개가 고대 유적을 보호하기 위해 출동한 것이다. 이탈리아 폼페이 고고학공원은 지난 28일 “로봇개 스폿이 폼페이 유적지 보호 임무에 투입됐다”고 밝혔다.',
          '미국 사우스웨스트 연구소의 켈시 싱어 박사 연구진은 29일(현지 시각) 국제 학술지 ‘네이처 커뮤니케이션’에 “미항공우주국(NASA·나사)의 명왕성 탐사선인 뉴허라이즌스가 관측한 명왕성의 두 산은 최근까지 얼음을 뿜어내는 화산 때문에 지금과 같은 불규칙적인 지형을 갖게 됐다”고 밝혔다.',
          '한국과 일본 과학자들이 바나나에 손상을 주지 않고 껍질만 깔 수 있는 로봇을 개발했다. 이전에는 힘 조절이 어려워 바나나처럼 부드러운 과일은 으깨버리기 일쑤였지만 이번 로봇은 인공지능 덕분에 사람처럼 유연한 동작을 할 수 있었다.',
          '인간이 술을 좋아하는 이유가 수백만 년 전 유인원 조상이 발효된 과일을 골라 먹던 데서 시작됐다는 ‘술 취한 원숭이’ 가설이 입증됐다. 원숭이가 알코올(에탄올)이 함유된 과일을 즐겨 먹고 소변에서도 알코올의 2차 대사물질이 검출돼 가설이 과학적으로 확인됐다는 것이다.']

acc = 0
for sentence in it_sci:
    if testModel(loaded_model, sentence) == 'IT/과학':
        acc += 1
print(f'\n정확도 : {acc*100 // len(it_sci)}%')

뉴스의 카테고리 : IT/과학 (신뢰도 99.99%)
뉴스의 카테고리 : IT/과학 (신뢰도 99.99%)
뉴스의 카테고리 : IT/과학 (신뢰도 99.99%)
뉴스의 카테고리 : IT/과학 (신뢰도 99.99%)
뉴스의 카테고리 : IT/과학 (신뢰도 99.99%)
뉴스의 카테고리 : 사회 (신뢰도 99.95%)
뉴스의 카테고리 : IT/과학 (신뢰도 99.99%)
뉴스의 카테고리 : IT/과학 (신뢰도 99.98%)
뉴스의 카테고리 : IT/과학 (신뢰도 99.99%)
뉴스의 카테고리 : IT/과학 (신뢰도 99.99%)

정확도 : 90%


In [56]:
#건강
health = ['40대 직장인 L씨는 몇 년 전부터 자신이 쓰던 화장품을 \'비건 화장품\'으로 조금씩 바꿔 왔다. 동물실험을 하거나 동물성 원료를 쓰지는 않았는지를 꼼꼼히 살핀다. L씨는 "화장품 동물실험 영상을 접한 뒤 크게 충격을 받고 제조 과정과 성분을 따지기 시작했다"며 "나의 즐거움을 위해 동물에게 불필요한 고통을 주는 제품을 쓰는 것은 윤리적이지 않다고 생각했다"고 말했다.',
          '옆머리가 제대로 정돈되지 않으면 자칫 지저분한 인상을 주기 쉽죠. 앞머리가 푹 꺼지면 피곤한 인상을 만들 수 있습니다. 옆머리는 바짝 눌러 깔끔하게 정돈하고, 앞머리는 가르마펌을 한 듯 풍성하게 볼륨을 살려 생기 있는 인상을 만드는 드라이 방법을 소개합니다.',
          'JARRET 이지연 디자이너가 지난 30일 서울 중구 장충단로 크레스트72에서 새롭게 론칭하는 하이앤드 패션브랜드 CEEANN과 컨템포러리 브랜드 CBCL의 F/W 2022 컬렉션을 발표했다.',
          '바닐라코는 카카오프렌즈의 인기 캐릭터인 리틀어피치와 협업한 ‘클린 잇 제로 리틀어피치 콜라보 기획 세트’를 선보인다고 1일 밝혔다. 리틀카카오프렌즈에서 가장 인기 있는 캐릭터 중 하나인 리틀어피치와 함께 한 이번 콜라보는 바닐라코의 감성과 리틀어피치의 사랑스럽고 트렌디한 이미지가 부합해 진행하게 됐다.',
          '손예진이 화보에서 선택한 브랜드는 ‘베라 왕’과 ‘엘리 사브’. 그 중 엘리 사브는 레바논 출신으로 파리 패션위크를 통해 오트 쿠튀르와 기성복 라인을 선보이는 유명 디자이너다. 파리 오트 쿠튀르 의상조합(Chambre Syndicale de la Haute Couture) 멤버이기도 하다. 로맨틱하면서도 여성스럽고 마치 요정들을 그대로 옮겨온 듯 판타지 동화 속으로 런웨이를 옮겨 놓는 디자인으로 유명하다.',
          '4월이 시작되면서 낮 기온이 올라가는 봄 날씨가 계속되고 있다. 산과 들에 꽃이 피고 하늘이 높고 파란, 완연한 봄기운이 느껴진다. 하지만 봄은 사람의 피부, 그중에서도 주름에 온화하지만은 않다. 이는 봄철에 따뜻해지는 날씨와 관련이 크다. 겨울 동안 차갑고 건조한 날씨에 익숙해져 있는 피부가 따뜻한 공기를 만나면 손상될 가능성이 크기 때문이다. 피부 손상에 따라 주름도 더욱 깊어질 수 있다.',
          '임신 중 의료기관 등에서 일하며 소독제를 자주 사용한 여성의 자녀는 세 살 때 천식 또는 아토피성 피부염이 발병할 위험이 높다는 연구 결과가 나왔다. 지난 30일(현지시간) UPI통신에 따르면 일본 야마나시(山梨) 대학의 고지마 레이지 보건과학 교수 연구팀이 \'환경과 아동 연구\' 참가 모자(母子) 7만8915쌍의 조사 자료를 분석한 결과 이 같은 사실이 밝혀졌다.',
          '맛있는 음식을 먹으면 사랑하는 존재가 떠오른다. 바로 우리 댕댕이! 하지만 사람 몸에 좋은 음식이 반려견 건강엔 치명적일 수 있다. 금지 음식물을 섭취하면 구토와 설사를 시작으로 장기 영구 손상이나 사망에 이를 수 있다. 반려견의 건강을 지키기 위해 함께 먹어선 \'안\' 되는 음식을 알아본다.',
          '"멀리서 오는 노인분의 걸음걸이가 이상하더군요. 유심히 봤더니 그냥 앞으로 고꾸라지셔서..." 수원의 한 아파트 단지의 경비원이 의식을 잃고 쓰러진 80대 주민을 심폐소생술로 살려 주목받고 있다. 이 경비원은 주민이 쓰러지자 곧바로 달려갔다. 긴급 출동한 소방서 구급대원은 "현장 도착 전에 경비원께서 응급 처치를 잘 해주셨다"고 말했다.',
          '손톱은 우리 몸의 건강 상태를 나타내주는 지표다. 손톱이 감싸고 있는 손끝에서 모세혈관이 모이기 때문에 특히 혈액순환과 관련된 문제를 파악하기 쉽다. 건강한 손톱은 투명한 분홍빛을 띤다. 손톱이 평소보다 진한 자줏빛이라면 혈액이 탁하고 순환이 잘 안 되는 것이다. 손톱이 하얗게 변할 때까지 눌렀다가 떼어냈을 때 분홍색으로 빨리 돌아오지 않으면 빈혈이나 혈액순환 장애의 가능성이 있다.']

acc = 0
for sentence in health:
    if testModel(loaded_model, sentence) == '건강':
        acc += 1
print(f'\n정확도 : {acc*100 // len(health)}%')

뉴스의 카테고리 : 건강 (신뢰도 99.96%)
뉴스의 카테고리 : 건강 (신뢰도 99.96%)
뉴스의 카테고리 : 건강 (신뢰도 99.97%)
뉴스의 카테고리 : 연예 (신뢰도 57.58%)
뉴스의 카테고리 : 건강 (신뢰도 99.89%)
뉴스의 카테고리 : 건강 (신뢰도 99.97%)
뉴스의 카테고리 : 건강 (신뢰도 99.97%)
뉴스의 카테고리 : 생활 (신뢰도 99.96%)
뉴스의 카테고리 : 건강 (신뢰도 99.96%)
뉴스의 카테고리 : 건강 (신뢰도 99.96%)

정확도 : 80%


In [89]:
#문화
culture = ['\'아트테크(아트+재테크)\'가 화두로 떠오른 가운데, 미술품에 투자하고 싶지만 성장 가능성이 높은 신진 작가를 알아볼 만한 안목은 아직 부족하고 유명 작가의 작품을 사기엔 높은 가격이 부담인 사람들이 \'아트 주식\'으로 눈을 돌리고 있다.',
          '서울 한복판에서 일상회복을 염원하는 희망의 음악 제전이 펼쳐진다. 한화그룹이 후원하는 ‘한화와 함께하는 2022 교향악축제’가 2일부터 24일까지 23일간 서울 예술의전당 콘서트홀에서 열린다. 올해는 우리나라를 대표하는 전국 20개 교향악단이 참여하고, 28명의 협연자와 총 20회 일정으로 꾸며진다.',
          '최근 몇 년 새 미국 동부와 서부를 넘어 유럽과 아시아까지 전 세계적 관심을 받는 작가가 있다. 마흔 살을 갓 넘겼고, LA에 기반을 둔 아티스트 힐러리 페시스(Hilary Pecis)다. 우리나라에선 지난 2020년 가나아트 사운즈 한남에서 열렸던 ‘리플렉션스: 오픈 엔디드’ 그룹전에서 소개된 적 있다.',
          '허용 여부를 놓고 논란이 많았던 태국 \'물 축제\' 송끄란 연휴 기간의 물싸움이 결국 금지됐다. 심상치 않은 코로나19 신규확진자 증가 사태가 고려된 것으로 보인다.',
          '여러분, 기사나 책을 통해 ‘예술로 아픔을 승화한다’는 말을 들어보셨을 겁니다. 문자로만 놓고 보면 조금 따분한 말이지요. 오늘 소개드릴 작가는 이 진부한 문장을 인생에 걸쳐 증명한 사람이라 할 수 있습니다. 프랑스 출신 대표 현대예술가 니키 드 생팔(1930~2002)인데요. 화려한 문양으로 뒤덮인 풍만한 여성상 ‘나나’로 유명한 작가입니다.',
          ' 봄날 자연에서 펼쳐지는 국내 유일 산악영화 축제인 제7회 울주세계산악영화제(www.umff.kr)가 개막 이틀째인 2일 첫 주말을 맞아 영화 상영과 더불어 다양한 프로그램이 선보인다. 영화는 이날 오전 10시부터 오후 9시까지 모두 14편이 상영된다.',
          '경기 포천에서 \'우드카빙\'을 즐기고, 배우고, 체험할 수 있는 전시회가 열린다. 포천 소흘읍에서 국내 유일의 우드카빙 국내외 공구·장비 판매장과 함께 교육장을 운영하고 있는 하우스플러스㈜ 우드워커스토어에서 우드카빙을 통해 힐링할 수 있는 전시회가 5~15일 펼쳐진다.',
          '방탄소년단은 이달 8~9일(이하 현지시간)과 15~16일 라스베이거스 얼리전트 스타디움(Allegiant Stadium)에서 ‘BTS 퍼미션 투 댄스 온 스테이지 - 라스베이거스’(BTS PERMISSION TO DANCE ON STAGE - LAS VEGAS)를 개최한다. 지난달 서울 잠실종합운동장 주경기장에서 열린 콘서트 이후 약 한 달 만에 대면 콘서트를 통해 다시 팬들을 만난다.',
          '강릉 능가사 주지 법관 스님이 선화(禪畵)의 전통을 이어온 개인전 \'선禪2022\'를 오는 5월1일까지 선보인다. 지난달 30일 서울 삼청동 학고재 갤러리에서 개막한 이번 전시에는 지난해부터 제작한 \'선\' 연작 42점을 비롯해 직접 빚은 다완과 족자 그림도 선보인다.',
          '부산 영화의전당이 2일부터 다음 달 29일까지 하늘연극장에서 극단 ‘신주쿠양산박’, 제작사 ‘㈜신주쿠양산박종로양산박’과 공동제작한 뮤지컬 ‘더 나우(THE NOW·포스터)’를 선보인다.']

acc = 0
for sentence in culture:
    if testModel(loaded_model, sentence) == '문화':
        acc += 1
print(f'\n정확도 : {acc*100 // len(culture)}%')

뉴스의 카테고리 : 문화 (신뢰도 99.94%)
뉴스의 카테고리 : 문화 (신뢰도 99.54%)
뉴스의 카테고리 : 문화 (신뢰도 99.96%)
뉴스의 카테고리 : 건강 (신뢰도 99.92%)
뉴스의 카테고리 : 문화 (신뢰도 99.87%)
뉴스의 카테고리 : 문화 (신뢰도 99.95%)
뉴스의 카테고리 : IT/과학 (신뢰도 99.99%)
뉴스의 카테고리 : 문화 (신뢰도 99.94%)
뉴스의 카테고리 : 문화 (신뢰도 98.6%)
뉴스의 카테고리 : 문화 (신뢰도 99.95%)

정확도 : 80%
